In [1]:
import pandas as pd
import numpy as np
import datetime
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import plotly.express as px

In [2]:
df = pd.read_csv('pick_pack_TBN1.csv')
df_fore = pd.read_csv('forecast_volume.csv')

In [3]:
df[['warehouse', 'day_type']] = df[['warehouse', 'day_type']].astype('category')
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df['datetime'] = ( pd.to_datetime(df['day'])  + pd.to_timedelta(df['hour'], unit='h'))
df = df.drop(columns=['day', 'hour'])

# 2) Create item_id before one-hot
df['item_id'] = df['warehouse'].astype(str) + '_' + df['day_type'].astype(str)

# # 3) One-hot encode
# df_encoded = pd.get_dummies(df, columns=['warehouse', 'day_type'], drop_first=True)

In [4]:
df_fore['day'] = pd.to_datetime(df_fore['full_date']).dt.date
df_fore['day'] = pd.to_datetime(df_fore['day'])
df_fore.sort_values(by='day', inplace=True)

df_fore['datetime'] = ( pd.to_datetime(df_fore['full_date'])  + pd.to_timedelta(df_fore['hour'], unit='h'))
df_fore[['warehouse','day_type']] = df_fore[['warehouse','day_type']].astype('category')

# Drop cột không cần
df_fore_processed = df_fore.drop(columns=['day', 'full_date', 'hour'])
df_fore_processed = df_fore_processed.rename(columns={'units_forecast': 'units', 'orders_forecast': 'orders'})

# Tạo item_id để group series
df_fore_processed['item_id'] = df_fore_processed['warehouse'].astype(str) + '_' + df_fore_processed['day_type'].astype(str)

# df_fore_enc = pd.get_dummies(df_fore_processed, columns=['warehouse','day_type'], drop_first=True)

df_fore_processed['count_pick'] = np.nan
df_fore_processed['count_pack'] = np.nan

In [5]:
# df_merged = pd.merge(
#     df_fore_processed,
#     df,
#     on=['datetime', 'item_id'],
#     how='left',
#     suffixes=('_fore', '_hist')
# )
# cutoff_date = pd.Timestamp('2025-05-15')
# df_merged['units']       = np.where(df_merged['datetime'] <= '2025-05-13 23:00:00', df_merged['units_hist'],       df_merged['units_fore'])
# df_merged['orders']      = np.where(df_merged['datetime'] <= '2025-05-13 23:00:00', df_merged['orders_hist'],      df_merged['orders_fore'])
# df_merged['count_pick']  = np.where(df_merged['datetime'] <= '2025-05-13 23:00:00', df_merged['count_pick_hist'],  np.nan)
# df_merged['count_pack']  = np.where(df_merged['datetime'] <= '2025-05-13 23:00:00', df_merged['count_pack_hist'],  np.nan)


# df_merged_enc = pd.get_dummies(df_merged, columns=['warehouse_fore', 'day_type_fore'], drop_first=True)

# df_merged_final = df_merged_enc[['datetime', 'item_id', 'units', 'orders', 'count_pick', 'count_pack']].copy()
# df_merged_final = df_merged_final.dropna(subset=['units', 'orders'])
# df_merged_train = df_merged_final[
#     df_merged_final['datetime'] <= cutoff_date
# ].dropna(subset=['count_pick', 'count_pack'])

# train_ts = TimeSeriesDataFrame.from_data_frame(
#     df_merged_train,
#     id_column="item_id",
#     timestamp_column="datetime"
# )

# prediction_pack = TimeSeriesPredictor.load("Autogluon_Pack_avg")
# prediction_pick = TimeSeriesPredictor.load("Autogluon_Pick_avg")

# pred_pack_ts = prediction_pack.predict(train_ts, random_seed=42).to_data_frame().reset_index()
# pred_pick_ts = prediction_pick.predict(train_ts, random_seed=42).to_data_frame().reset_index()


# df_pack_pred = pred_pack_ts.rename(columns={'mean': 'count_pack'})
# df_pick_pred = pred_pick_ts.rename(columns={'mean': 'count_pick'})
# df_pred = pd.merge(df_pack_pred, df_pick_pred, on=['item_id', 'timestamp'], how='outer')
# df_pred = df_pred.rename(columns={'timestamp': 'datetime'})

# df_pred.head()

In [6]:
# df_merged_combined = pd.merge(
#     df_merged_final,
#     df_pred[['item_id', 'datetime', 'count_pack', 'count_pick']],
#     on=[ 'datetime'],
#     how='outer',  # giữ tất cả dữ liệu từ cả hai bảng
#     suffixes=('', '_forecast')
# )

# # Nếu muốn dùng forecast làm giá trị thay thế khi thiếu
# df_merged_combined['count_pack'] = df_merged_combined['count_pack'].combine_first(df_merged_combined['count_pack_forecast'])
# df_merged_combined['count_pick'] = df_merged_combined['count_pick'].combine_first(df_merged_combined['count_pick_forecast'])

# # Xoá cột forecast tạm
# df_merged_combined.drop(columns=['count_pack_forecast', 'count_pick_forecast'], inplace=True)


In [7]:
# 1. Merge forecast và history ban đầu
df_merged = pd.merge(
    df_fore_processed,
    df,
    on=['datetime', 'item_id'],
    how='left',
    suffixes=('_fore', '_hist')
)

# 2. Kiểm tra cột bắt buộc
required_cols = ['units_fore', 'units_hist', 'orders_fore', 'orders_hist',
                 'count_pick_hist', 'count_pack_hist']
missing_cols = [col for col in required_cols if col not in df_merged.columns]
if missing_cols:
    raise ValueError(f"Thiếu cột sau merge: {missing_cols}")

# 3. Cập nhật theo cutoff đầu tiên (ngày cuối cùng có lịch sử)
cutoff_base = pd.Timestamp("2025-05-13 23:00:00")
df_merged['units']      = np.where(df_merged['datetime'] <= cutoff_base, df_merged['units_hist'], df_merged['units_fore'])
df_merged['orders']     = np.where(df_merged['datetime'] <= cutoff_base, df_merged['orders_hist'], df_merged['orders_fore'])
df_merged['count_pick'] = np.where(df_merged['datetime'] <= cutoff_base, df_merged['count_pick_hist'], np.nan)
df_merged['count_pack'] = np.where(df_merged['datetime'] <= cutoff_base, df_merged['count_pack_hist'], np.nan)

# 4. Mã hóa one-hot nếu cần
df_merged_enc = pd.get_dummies(df_merged, columns=['warehouse_fore', 'day_type_fore'], drop_first=True)

# 5. Lọc các cột cần dùng
df_merged_final = df_merged_enc[['datetime', 'item_id', 'units', 'orders', 'count_pick', 'count_pack']].copy()
df_merged_final = df_merged_final.dropna(subset=['units', 'orders'])

# Lưu kết quả dự báo
all_forecasts = []

# 6. Vòng lặp 7 ngày
for i in range(15):
    forecast_date = pd.Timestamp('2025-05-14') + pd.Timedelta(days=i)
    forecast_start = forecast_date
    forecast_end = forecast_date + pd.Timedelta(days=1)

    # Tạo tập huấn luyện đến thời điểm hiện tại
    df_merged_train = df_merged_final[df_merged_final['datetime'] <= forecast_date - pd.Timedelta(hours=1)]
    df_merged_train = df_merged_train.dropna(subset=['count_pick', 'count_pack'])

    if df_merged_train.empty:
        print(f"⛔️ Không đủ dữ liệu để train cho ngày {forecast_date.date()}, bỏ qua.")
        continue

    # Tạo TimeSeriesDataFrame
    train_ts = TimeSeriesDataFrame.from_data_frame(
        df_merged_train,
        id_column="item_id",
        timestamp_column="datetime"
    )

    # Load model & predict
    prediction_pack = TimeSeriesPredictor.load("Autogluon_Pack_avg")
    prediction_pick = TimeSeriesPredictor.load("Autogluon_Pick_avg")

    pred_pack_ts = prediction_pack.predict(train_ts, random_seed=42).to_data_frame().reset_index()
    pred_pick_ts = prediction_pick.predict(train_ts, random_seed=42).to_data_frame().reset_index()

    df_pack_pred = pred_pack_ts.rename(columns={'mean': 'count_pack'})
    df_pick_pred = pred_pick_ts.rename(columns={'mean': 'count_pick'})
    df_pred = pd.merge(df_pack_pred, df_pick_pred, on=['item_id', 'timestamp'], how='outer')
    df_pred = df_pred.rename(columns={'timestamp': 'datetime'})

    # Lọc forecast đúng khung thời gian
    df_fc = df_pred[(df_pred['datetime'] >= forecast_start) & (df_pred['datetime'] < forecast_end)]

    if df_fc.empty:
        print(f"⚠️ Không có forecast cho ngày {forecast_date.date()}, bỏ qua.")
        continue

    # ✅ Gộp vào df_merged_final kiểu outer, dùng combine_first
    df_merged_final = pd.merge(
        df_merged_final,
        df_fc[['item_id', 'datetime', 'count_pack', 'count_pick']],
        on=['item_id', 'datetime'],
        how='outer',
        suffixes=('', '_forecast')
    )

    df_merged_final['count_pack'] = df_merged_final['count_pack'].combine_first(df_merged_final['count_pack_forecast'])
    df_merged_final['count_pick'] = df_merged_final['count_pick'].combine_first(df_merged_final['count_pick_forecast'])

    df_merged_final.drop(columns=['count_pack_forecast', 'count_pick_forecast'], inplace=True)

    print(f"✅ Dự báo xong ngày {forecast_date.date()} — {len(df_fc)} bản ghi.")
    all_forecasts.append(df_fc)

# 7. Gộp kết quả dự báo
if all_forecasts:
    df_all_forecasts = pd.concat(all_forecasts, ignore_index=True)
    df_all_forecasts = df_all_forecasts.sort_values(by=['item_id', 'datetime']).reset_index(drop=True)
else:
    df_all_forecasts = pd.DataFrame(columns=['item_id', 'datetime', 'count_pack', 'count_pick'])

# 8. Hiển thị kết quả đầu ra
df_all_forecasts.head(48)


data with frequency 'IRREG' has been resampled to frequency 'h'.
data with frequency 'IRREG' has been resampled to frequency 'h'.
data with frequency 'IRREG' has been resampled to frequency 'h'.


✅ Dự báo xong ngày 2025-05-14 — 23 bản ghi.


data with frequency 'IRREG' has been resampled to frequency 'h'.
data with frequency 'IRREG' has been resampled to frequency 'h'.


✅ Dự báo xong ngày 2025-05-15 — 23 bản ghi.


data with frequency 'IRREG' has been resampled to frequency 'h'.
data with frequency 'IRREG' has been resampled to frequency 'h'.


...

data with frequency 'IRREG' has been resampled to frequency 'h'.
data with frequency 'IRREG' has been resampled to frequency 'h'.


✅ Dự báo xong ngày 2025-05-27 — 23 bản ghi.


data with frequency 'IRREG' has been resampled to frequency 'h'.


✅ Dự báo xong ngày 2025-05-28 — 23 bản ghi.


,item_id,datetime,count_pack,0.1_x,0.5_x,0.9_x,count_pick,0.1_y,0.5_y,0.9_y
0,TBN1_Normal,2025-05-14 00:00:00,5.641587,0.437787,5.636020,9.041194,2.379339,0.650609,2.379306,4.084045
1,TBN1_Normal,2025-05-14 01:00:00,6.052952,0.770399,6.040912,9.640804,3.828759,1.921720,3.826703,5.688512
2,TBN1_Normal,2025-05-14 02:00:00,7.239376,1.651664,7.205642,11.276901,3.502360,1.518091,3.501970,5.426108
3,TBN1_Normal,2025-05-14 03:00:00,6.893043,1.447456,6.871487,11.004420,2.774519,0.543634,2.766988,4.963762
4,TBN1_Normal,2025-05-14 04:00:00,8.155748,2.839993,8.157876,12.368245,2.615007,0.025947,2.525837,5.276367
5,TBN1_Normal,2025-05-14 05:00:00,8.456751,3.810687,8.452527,12.215391,3.950608,1.977214,3.846508,5.982568
6,TBN1_Normal,2025-05-14 06:00:00,8.948272,4.473078,8.952974,12.714292,7.070139,4.278069,7.150101,9.599209
7,TBN1_Normal,2025-05-14 07:00:00,9.622400,5.693037,9.633430,12.973222,8.067960,5.326060,8.268063,10.760945
8,TBN1_Normal,2025-05-14 08:00:00,10.128787,5.914733,10.130705,13.853960,7.666685,3.701526,7.699147,10.910821
9,TBN1_Normal,2025-05-14 09:00:00,10.777164,6.157448,10.781615,14.964552,6.230510,2.928947,6.061461,9.787983


In [ ]:
# 1. Chuẩn bị FTE từ dự báo
FTE = df_all_forecasts[['item_id', 'datetime', 'count_pack', 'count_pick']].copy()
FTE['count_pack'] = np.ceil(FTE['count_pack'])
FTE['count_pick'] = np.ceil(FTE['count_pick'])
FTE['FTE_count'] = FTE['count_pack'] + FTE['count_pick']

# 2. Định nghĩa shift
shift_map = {
    'TKN_CA 26': ('12:00:00', '20:00:00'),
    'TKN_CA 1':  ('06:00:00', '14:00:00'),
    'TKN_CA 34': ('14:00:00', '22:00:00'),
    'TKN_CA 30': ('13:00:00', '21:00:00'),
    'TKN_CA 53': ('08:00:00', '16:00:00'),
    'TKN_CA 13': ('09:00:00', '17:00:00'),
    'TKN_CA 25': ('11:30:00', '19:30:00'),
}

# 3. Chuyển giờ sang dạng time
shift_times = {
    shift: (datetime.time.fromisoformat(start), datetime.time.fromisoformat(end))
    for shift, (start, end) in shift_map.items()
}

# 4. Tạo timestamp
FTE['timestamp'] = pd.to_datetime(FTE['datetime'])

# 5. Hàm gán shift
def assign_shift(ts):
    t = ts.time()
    for shift, (start, end) in shift_times.items():
        if start <= t < end:
            return shift
    return None

# 6. Gán shift và ngày
FTE['shift'] = FTE['timestamp'].apply(assign_shift)
FTE['date'] = FTE['timestamp'].dt.date

# 7. Tổng hợp theo item_id, ngày, ca
agg_result = (
    FTE.dropna(subset=['shift'])
       .groupby(['item_id', 'date', 'shift'], as_index=False)
       .agg({
           'count_pack': 'min',
           'count_pick': 'min',
           'FTE_count': 'min'
       })
)

# Làm tròn FTE_count
agg_result['FTE_count'] = np.ceil(agg_result['FTE_count'])

# 8. Xuất ra file CSV
agg_result.to_csv("FTE_forecast.csv", index=False)

# 9. Xem trước kết quả
print(agg_result.head(10))


       item_id        date      shift  count_pack  count_pick  FTE_count
0  TBN1_Normal  2025-05-14   TKN_CA 1         9.0         7.0       17.0
1  TBN1_Normal  2025-05-14  TKN_CA 26         9.0         6.0       15.0
2  TBN1_Normal  2025-05-14  TKN_CA 34         9.0         7.0       16.0
3  TBN1_Normal  2025-05-15   TKN_CA 1         9.0         7.0       16.0
4  TBN1_Normal  2025-05-15  TKN_CA 26         8.0         6.0       14.0
5  TBN1_Normal  2025-05-15  TKN_CA 34         9.0         6.0       15.0
6  TBN1_Normal  2025-05-16   TKN_CA 1         9.0         6.0       15.0
7  TBN1_Normal  2025-05-16  TKN_CA 26         9.0         6.0       15.0
8  TBN1_Normal  2025-05-16  TKN_CA 34         9.0         6.0       15.0
9  TBN1_Normal  2025-05-17   TKN_CA 1         9.0         6.0       16.0


In [55]:
# Giả sử biến bạn đang dùng là result
fig = px.imshow(
    result.pivot(index='shift', columns='date', values='FTE_count'),
    labels=dict(x="Date", y="Shift", color="FTE Count"),
    color_continuous_scale='Blues',
    text_auto=".2f"  # hiển thị số ngay trong ô
)

fig.update_layout(
    title="Shift Forecast",
    xaxis_title="Date",
    yaxis_title="Shift",
    xaxis=dict(type='category'),
    yaxis=dict(type='category'),
    autosize=True
)

fig.show()
